In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import pickle
MODELS_PATH = r'..\models'
PROCESSED_DATASET_PATH = r"..\dataset_processed"

In [ ]:
with open(rf"{PROCESSED_DATASET_PATH}\functional_features.pkl", "rb") as input_file:
    X = pickle.load(input_file)
le = LabelEncoder()
Y = le.fit_transform(X['Emotion'])
X = X.drop(['Emotion','filepath'], axis=1)

In [ ]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
with open(rf"{MODELS_PATH}\xgb_scaler.pkl", "wb") as output_file:
    pickle.dump([le, scaler], output_file)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
xgb_model = xgb.XGBClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [2, 4, 6,8],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}
grid = RandomizedSearchCV(xgb_model, param_grid, refit=True, verbose=1,cv=3,n_iter=15,random_state=42)
grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
with open(rf"{MODELS_PATH}\svm_model.pkl", "wb") as output_file:
    pickle.dump(grid, output_file)